In [2]:
import openai
import langchain
from langchain.llms import OpenAI

In [4]:
OPENAI_KEY=""

In [22]:
client = OpenAI(openai_api_key=OPENAI_KEY)

ValidationError: 1 validation error for OpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [6]:
prompt= "can you tell me total number of countries in aisa? can you give me top 10 country name?"

In [ ]:
print(client.predict(prompt).strip())

In [ ]:
prompt2 = "can you tell me capital of India?"

In [ ]:
client.predict(prompt2).strip()

## Prompt templates

In [7]:
from langchain.prompts import PromptTemplate

In [16]:
prompt_template_name=PromptTemplate(
    input_variables=["country"],
    template="can you tell me the capital of {country}?"
    )

In [17]:
prompt1=prompt_template_name.format(country='india')

In [18]:
prompt2=prompt_template_name.format(country='china')

In [ ]:
client.predict(prompt1).strip()

In [19]:
# second way to create prompt template
prompt = PromptTemplate.from_template("What is a good name company that makes a {product}")

In [20]:
prompt3 = prompt.format(product='toys')

In [23]:
client.predict(prompt3).strip()

NameError: name 'client' is not defined

## Agent

We use this agent in Langchain for calling any third party tool
Inorder to extract the real time info I am going to use serp api

now by using 'serp api' i will call google-search-engine

and i will extract the info in a real time

In [24]:
prompt4 = "Can you tell me who won the recent cricket world cup?"

In [ ]:
client.predict(prompt4).strip()

In [25]:
prompt5 = "Can you tell me the current GDP of india?"

In [ ]:
client.predict(prompt5).strip()

In [26]:
! pip install google-search-results

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32077 sha256=9d89ecfa741b89f337c66688268eeff53e4b39a947dc7df00539ecd0278b1a8c
  Stored in directory: c:\users\sowmy\appdata\local\pip\cache\wheels\68\8e\73\744b7d9d7ac618849d93081a20e1c0deccd2aef90901c9f5a9
Successfully built google-search-results


In [27]:
serpapi_key = "778f3a0d1c9450431a502601213f70c3bfd7f0449241bcf67ff128f72337b556"

In [28]:
from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

In [ ]:
client=OpenAI(openai_api_key=OPENAI_KEY)

In [ ]:
tool=load_tools(["serpapi"], serpapi_key=serpapi_key, llm=client)

In [ ]:
agent = initialize_agent(tool, client, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [ ]:
agent.run("Can you tell me who won the recent cricket world cup?")

In [ ]:
agent.run("can you tell me 5 top current affairs?")

In [32]:
! pip install wikipedia

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
Using cached soupsieve-2.5-py3-none-any.whl (36 kB)
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11704 sha256=782074c5cb2fb612baf0d478ad37c0f4ce6a0869bbdb4e584977a15afc9ce0dc
  Stored in directory: c:\users\sowmy\appdata\local\pip\cache\wheels\c2\46\f4\caa1bee71096d7b0cdca2f2a2af45cacf35c5760bee8f00948
Successfully built wikipedia


In [ ]:
tool = load_tools(["wikipedia"], llm=client)

In [ ]:
agent = initialize_agent(tool, client, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("Can you tell me about recent cricket worldcup?")

In [ ]:
agent.run("Can you tell me about GDP of USA?")

## Chain

In [1]:
from langchain.prompts import PromptTemplate

In [2]:
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}")

In [4]:
from langchain.chains import LLMChain

In [ ]:
chain=LLMChain(llm=client, prompt=prompt)

In [ ]:
chain.run("Wine").strip()

##### Example 2

In [5]:
prompt_template=PromptTemplate(
    input_variables=["cuisine"],
    template = "I want to open a restuarent for {cuisine} food, suggest a fancy name for this"
    )

In [6]:
prompt_template

PromptTemplate(input_variables=['cuisine'], template='I want to open a restuarent for {cuisine} food, suggest a fancy name for this')

In [7]:
chain=LLMChain(llm=client, prompt=prompt_template)

NameError: name 'client' is not defined

In [ ]:
chain.run("indian").strip()

In [ ]:
chain=LLMChain(llm=client, prompt=prompt_template, verbose=True)

In [ ]:
chain.run("American")

#### If we want to combine multiple chain and set a sequence for that we use simple sequential chain

In [ ]:
prompt_template_name = PromptTemplate(
    input_variables=["startup_name"],
    template="I want to start a startup for {startup-name}, suggest me a good name for this"
)

name_chain = LLMChain(llm=client, prompt=prompt_template_name)

In [ ]:
prompt_template_items=PromptTemplate(
    input_variables=["name"],
    template="suggest some strategies for {name}"
)

strategies_chain = LLMChain(llm=client, prompt=prompt_template_items)

In [ ]:
chain=SimpleSequentialChain(chains=[name_chain, strategies_chain])

In [ ]:
print(chain.run("artificial intelligence"))

### Now lets try to understand the sequential chain

In [ ]:
prompt_template_name=PromptTemplate(
    input_variables=["cuisine"],
    template = "I want to open a restuarent for {cuisine} food, suggest a fancy name for this"
    )

name_chain = LLMChain(llm=client, prompt=prompt_template_name, output_key="restaurant_name")

In [ ]:
prompt_template_items=PromptTemplate(
    input_variables=["restaurant_name"],
    template="suggest some menu items for {restaurant_name}"
)

food_items_chain = LLMChain(llm=client, prompt=prompt_template_items, output_key="menu_items")

In [10]:
from langchain.chains import SequentialChain

In [ ]:
chain=SequentialChain(chains=[name_chain, food_items_chain],
                      input_variables=["cuisine"],
                      output_variables=["restaurant_name", "menu_items"]
                      )

In [ ]:
chain({"cuisine": "indian"})

## Document Loader

In [11]:
! pip install pypdf

   ---------------------------------------- 0.0/278.2 kB ? eta -:--:--
   - -------------------------------------- 10.2/278.2 kB ? eta -:--:--
   ----------- ---------------------------- 81.9/278.2 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 278.2/278.2 kB 2.4 MB/s eta 0:00:00


In [12]:
from langchain.document_loaders import PyPDFLoader

In [15]:
loader = PyPDFLoader(r"D:\\Office_Docs\\Attra\\Payslip_Jan.pdf")

In [16]:
pages=loader.load_and_split()

In [17]:
pages

[Document(page_content='Attra Infotech Private LimitedPayslip For : JAN-2021 \nAmount in INR \nPersonnel No. 323760 Name SOWMYA A M\nBank HDFC Bank A/c No. 50100036421568\nDOJ 15 Jan 2019 Location BANGALORE\nPF No. PYBOM00349130000013750 Father Name MANJUNATHA A\nDepartment US Designation Senior Test Engineer\nPF UAN 100713520182 Band B\nBank IFSC CODE  PAN EOBPS2976Q\nDOB 27 Feb 1990 ESI number  \nSTD Days 31 Worked Days 31\nLOP Days 0   \nEARNINGS AMOUNT DEDUCTIONS AMOUNT\nBasic  36,350.00 Provident Fund  4,362.00\nHouse Rent  14,540.00 Professional Tax  200.00\nFBP - Taxable Portion  45,576.00 INCOME TAX  5,515.00\nLTC  3,028.00   \nGROSS EARNINGS 99,494.00 GROSS DEDUCTIONS 10,077.00\nNET PAY 89,417.00\nLeave Type Opening Balance Credit or Earned Availed Closing Balance\nPL 19.00 2.00 3.00 18.00\nAccured Leave 0.00 NA 0.00 0.00\n** This is a computer generated payslip and does not require signature and stamp.', metadata={'source': 'D:\\\\Office_Docs\\\\Attra\\\\Payslip_Jan.pdf', 'pa

## Memory

In [2]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

In [ ]:
client = OpenAI(openai_api_key=OPENAI_KEY)

In [ ]:
prompt_template_name=PromptTemplate(
    input_variables=['product'],
    template="What is a good name for a company that makes {product}"
)

In [ ]:
chain = LLMChain(llm=client, prompt=prompt_template_name)

In [ ]:
chain.run('colourful cup').strip()

In [ ]:
prompt_template_name=PromptTemplate(
    input_variables=['product'],
    template="What is a good name for a company that makes {product}"
)

In [ ]:
chain = LLMChain(llm=client, prompt=prompt_template_name)

In [ ]:
chain.run('drons').strip()

In [ ]:
chain.memory

In [ ]:
type(chain.memory)

## ConversationBufferMemory
We can attach memory to remember all previous conversation

In [4]:
from langchain.memory import ConversationBufferMemory

In [5]:
memory = ConversationBufferMemory()

In [6]:
prompt_template_name=PromptTemplate(
    input_variables=['product'],
    template="What is a good name for a company that makes {product}"
)

In [ ]:
chain=LLMChain(llm=client, prompt=prompt_template_name, memory=memory)

In [ ]:
chain.run("wines")

In [ ]:
chain.run("camera")

In [ ]:
chain.run("drone")

In [ ]:
chain.memory

In [ ]:
print(chain.memory.buffer)

### ConversationChain

- Conversation buffer memory goes growing endlessly
- just rememver last 5 Conversation Chain
- Just rememeber last 10-20 conversation chain

In [8]:
from langchain.chains import ConversationChain

In [ ]:
convo=ConversationChain(llm=OpenAI(openai_api_key=OPENAI_KEY, temperature=0.7))

In [ ]:
convo.prompt

In [ ]:
print(convo.prompt.template)

In [ ]:
convo.run("who won the first cricket world cut?")

In [ ]:
convo.run("can you tell me5+5?")

In [ ]:
convo.run("can you tell me how much will 5*(5+%)")

In [ ]:
convo.run("who was the captain of the winning team?")

In [ ]:
convo.run("Can you divde the numbers and can you give me the final answer")

### ConversationBufferWindowMemory

In [10]:
from langchain.memory import ConversationBufferWindowMemory

In [11]:
memory = ConversationBufferWindowMemory(k=1)
# based of k value it will keep the memory of the prompts
# if we didn't provide k avalue, then it will track all the values

In [ ]:
convo=ConversationChain(llm=OpenAI(openai_api_key=OPENAI_KEY, temparature=0.7), memory=memory)

In [ ]:
convo.run("who won the cricket first worldcup?")

In [ ]:
convo.run("can you tell how much will 5+%?")

In [ ]:
convo.run("who was the captain of the winning team?")